In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%gui qt5

In [3]:
import napari

In [4]:
from pathlib import Path
import time
import pandas as pd

In [5]:
from pysmFISH.data_models import Dataset
from pysmFISH.stitching import organize_square_tiles
from pysmFISH.utils import create_dir
from pysmFISH.stitching import stitching_graph,stitch_using_coords_general,stitching_graph_fresh_nuclei
from pysmFISH import io


In [6]:
from dask.distributed import LocalCluster
from dask.distributed import Client

In [7]:
cluster = LocalCluster(n_workers=2, memory_limit='6GB', processes=True,threads_per_worker=1)
client = Client(cluster)

/Users/simone.codeluppi/opt/miniconda3/envs/test_d/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60730 instead
  warnings.warn(


In [8]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:60730/status,
Dashboard: http://127.0.0.1:60730/status,Workers: 2
Total threads: 2,Total memory: 11.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60731,Workers: 2
Dashboard: http://127.0.0.1:60730/status,Total threads: 2
Started: Just now,Total memory: 11.18 GiB
Comm: tcp://127.0.0.1:60743,Total threads: 1
Dashboard: http://127.0.0.1:60744/status,Memory: 5.59 GiB
Nanny: tcp://127.0.0.1:60734,


In [19]:
def stitched_beads_on_nuclei_fresh_tissue(experiment_fpath,
                                      client,
                                      nuclei_tag='_ChannelCy3_Nuclei_',
                                      beads_tag='_ChannelEuropium_Cy3_',
                                      round_num = 1,
                                      overlapping_percentage=5,
                                      machine='ROBOFISH2'
                                     ):
    experiment_fpath = Path(experiment_fpath)
    fresh_tissue_path = experiment_fpath / 'fresh_tissue'
    beads_dataset_fpath = list(fresh_tissue_path.glob('*'+ beads_tag +'*.parquet'))[0]
    nuclei_dataset_fpath = list(fresh_tissue_path.glob('*'+ nuclei_tag +'*.parquet'))[0]
    
    # Collect and adjust beads dataset with missing values
    beads_data = Dataset()
    beads_data.load_dataset(beads_dataset_fpath)
    beads_data.dataset['processing_type'] = 'undefined'
    beads_data.dataset['overlapping_percentage'] = overlapping_percentage / 100
    beads_data.dataset['machine'] = machine

    metadata_beads = beads_data.collect_metadata(beads_data.dataset)
    beads_org_tiles = organize_square_tiles(experiment_fpath,beads_data.dataset,metadata_beads,round_num)
    beads_org_tiles.run_tiles_organization()
    flist = list((fresh_tissue_path / 'results').glob('*decoded_fov*.parquet'))

    # duplicate registered
    for fpath in flist:
        data = pd.read_parquet(fpath)
        data['r_px_registered'] = data['r_px_original']
        data['c_px_registered'] = data['c_px_original']
        data['hamming_distance'] = 0
        data['decoded_genes'] = 'beads'
        data.to_parquet(fpath)


    all_futures = []
    for fpath in flist:
        future = client.submit(stitch_using_coords_general, fpath,
                                              beads_org_tiles.tile_corners_coords_pxl,
                                              beads_org_tiles.reference_corner_fov_position,
                                              metadata_beads,tag='microscope_stitched')

        all_futures.append(future)
    _ = client.gather(all_futures)

    io.simple_output_plotting(fresh_tissue_path,
                              stitching_selected= 'microscope_stitched',
                             selected_Hdistance=0,
                             client = client,
                             input_file_tag = 'decoded_fov',
                             file_tag = 'stitched_microscope')


    # Collect and adjust nuclei dataset with missing values
    nuclei_data = Dataset()
    nuclei_data.load_dataset(nuclei_dataset_fpath)
    nuclei_data.dataset['processing_type'] = 'undefined'
    nuclei_data.dataset['overlapping_percentage'] = overlapping_percentage / 100
    nuclei_data.dataset['machine'] = machine

    metadata_nuclei = nuclei_data.collect_metadata(nuclei_data.dataset)
    nuclei_org_tiles = organize_square_tiles(experiment_fpath,nuclei_data.dataset,metadata_nuclei,round_num)
    nuclei_org_tiles.run_tiles_organization()

    _ =stitching_graph_fresh_nuclei(experiment_fpath,nuclei_org_tiles, metadata_nuclei, 
                                client, nr_dim = 2)


    io.simple_output_plotting(fresh_tissue_path,
                              stitching_selected= 'global_stitched_nuclei',
                             selected_Hdistance=0,
                             client = client,
                             input_file_tag = 'decoded_fov',
                             file_tag = 'global_stitched_nuclei')

In [18]:
stitched_beads_on_nuclei('processing_folder/LBEXP20210718_EEL_Mouse_448_2/',
                                      client,
                                      nuclei_tag='_ChannelCy3_Nuclei_',
                                      beads_tag='_ChannelEuropium_Cy3_',
                                      round_num = 1,
                                      overlapping_percentage=5,
                                      machine='ROBOFISH2'
                                     )

In [9]:
experiment_fpath = Path('processing_folder/LBEXP20210718_EEL_Mouse_448_2/')
fresh_tissue_path = experiment_fpath / 'fresh_tissue'
beads_dataset_name = '211014_09_56_52_20210718_124116_843__ChannelEuropium_Cy3_Seq0002_img_data_dataset.parquet'
nuclei_dataset_name = '211014_09_56_53_20210718_124116_843__ChannelCy3_Nuclei_Seq0001_img_data_dataset.parquet'
round_num = 1
overlapping_percentage = 5

In [12]:
# Collect and adjust beads dataset with missing values
beads_data = Dataset()
beads_data.load_dataset(fresh_tissue_path / beads_dataset_name)
beads_data.dataset['processing_type'] = 'undefined'
beads_data.dataset['overlapping_percentage'] = 5 / 100
beads_data.dataset['machine'] = 'ROBOFISH2'

metadata_beads = beads_data.collect_metadata(beads_data.dataset)
beads_org_tiles = organize_square_tiles(experiment_fpath,beads_data.dataset,metadata_beads,round_num)
beads_org_tiles.run_tiles_organization()
flist = list((fresh_tissue_path / 'results').glob('*decoded_fov*.parquet'))

# duplicate registered
for fpath in flist:
    data = pd.read_parquet(fpath)
    data['r_px_registered'] = data['r_px_original']
    data['c_px_registered'] = data['c_px_original']
    data['hamming_distance'] = 0
    data['decoded_genes'] = 'beads'
    data.to_parquet(fpath)

    
all_futures = []
for fpath in flist:
    future = client.submit(stitch_using_coords_general, fpath,
                                          beads_org_tiles.tile_corners_coords_pxl,
                                          beads_org_tiles.reference_corner_fov_position,
                                          metadata_beads,tag='microscope_stitched')

    all_futures.append(future)
_ = client.gather(all_futures)

io.simple_output_plotting(fresh_tissue_path,
                          stitching_selected= 'microscope_stitched',
                         selected_Hdistance=0,
                         client = client,
                         input_file_tag = 'decoded_fov',
                         file_tag = 'stitched_microscope')


# Collect and adjust nuclei dataset with missing values
nuclei_data = Dataset()
nuclei_data.load_dataset(fresh_tissue_path / nuclei_dataset_name)
nuclei_data.dataset['processing_type'] = 'undefined'
nuclei_data.dataset['overlapping_percentage'] = 5 / 100
nuclei_data.dataset['machine'] = 'ROBOFISH2'

metadata_nuclei = nuclei_data.collect_metadata(nuclei_data.dataset)
nuclei_org_tiles = organize_square_tiles(experiment_fpath,nuclei_data.dataset,metadata_nuclei,round_num)
nuclei_org_tiles.run_tiles_organization()

_ =stitching_graph_fresh_nuclei(experiment_fpath,nuclei_org_tiles, metadata_nuclei, 
                            client, nr_dim = 2)


io.simple_output_plotting(fresh_tissue_path,
                          stitching_selected= 'global_stitched_nuclei',
                         selected_Hdistance=0,
                         client = client,
                         input_file_tag = 'decoded_fov',
                         file_tag = 'global_stitched_nuclei')

In [ ]:
nuclei_org_tiles.overlapping_order

In [ ]:
io.simple_output_plotting(experiment_fpath,
                          stitching_selected= 'global_stitched_nuclei',
                         selected_Hdistance=0,
                         client = client,
                         input_file_tag = 'decoded_fov',
                         file_tag = 'global_stitched_nuclei')

In [ ]:
stitching_channel = 'Europium'
reference_round = 1
stitching_graph(experiment_fpath, stitching_channel,beads_org_tiles, metadata, 
                    reference_round, client, nr_dim = 2)

In [ ]:
_ =stitching_graph_fresh_nuclei(experiment_fpath,beads_org_tiles, metadata, 
                            client, nr_dim = 2)

In [ ]:
data = pd.read_parquet(fpath)

In [ ]:
from pysmFISH.pipeline import Pipeline

In [ ]:
# Enter the required parameters
experiment_fpath = Path('processing_folder/LBEXP20210718_EEL_Mouse_448_2')
dataset_name = '210906_21_07_09_LBEXP20210718_EEL_Mouse_448_2_img_data_dataset.parquet'
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'local'

In [ ]:
%%time
# Because you are running the pipeline locally you should define the number of cores and memory/core
# 3-4 GB / code should be safe for running the processing
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        cores=3,
        memory='12GB',
        chunk_size = 6,
        dataset_path = experiment_fpath / dataset_name)

In [ ]:
%%time
running_pipeline.run_parsing_only()

In [ ]:
%%time
running_pipeline.run_required_steps()

In [ ]:
running_pipeline.client

In [ ]:
running_pipeline.stitch_and_remove_dots_eel_graph_step()

In [ ]:
running_pipeline.client.close()

In [ ]:
running_pipeline.cluster.close()

# Visualize stitched counts

In [ ]:
from dask import dataframe as dd

In [ ]:
experiment_fpath = Path('processing_folder/LBEXP20210718_EEL_Mouse_448_2/fresh_tissue/')
stitched_40X_beads_pd = pd.read_parquet(experiment_fpath / 'results' / '211014_11_59_25_fresh_tissue_data_summary_simple_plotting_stitched_microscope.parquet')
stitched_40X_beads_pd = stitched_40X_beads_pd.dropna()

In [ ]:
stitched_40X_coords = stitched_40X_beads_pd.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()

In [ ]:
vw = napari.Viewer()
_ = vw.add_points(stitched_40X_coords,face_color='magenta', symbol='o', size= 20)

In [ ]:
exp_fpath = Path('processing_folder/LBEXP20210718_EEL_Mouse_448_2/')
data_dd =dd.read_parquet(exp_fpath /'results' / '*_decoded_fov_*')

In [ ]:
beads_60X_df = data_dd.loc[(data_dd.channel=='Europium') &
                      (data_dd.mapped_beads_type =='large'), ['r_px_microscope_stitched','c_px_microscope_stitched'] ].compute()

In [ ]:
beads_60X_df.to_parquet(exp_fpath / 'results' / 'large_beads_registered.parquet')

In [ ]:
beads_60X_coords = beads_60X_df.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()

In [ ]:
vw = napari.Viewer()
_ = vw.add_points(stitched_40X_coords,name='40X',face_color='magenta', symbol='o', size= 120)
_ = vw.add_points(beads_60X_coords,name='60X',face_color='green', symbol='+', size= 120)

In [ ]:
import open3d as o3d

In [ ]:
# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
xyz = np.c_[stitched_40X_coords,np.ones(stitched_40X_coords.shape[0]) ]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
o3d.io.write_point_cloud((exp_fpath.as_posix() + '/40X_converted.ply'), pcd)

In [ ]:
# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
xyz = np.c_[beads_60X_coords,np.ones(beads_60X_coords.shape[0]) ]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
o3d.io.write_point_cloud((exp_fpath.as_posix() + '/60X_converted.ply'), pcd)

In [ ]:
o3d.visualization.draw_geometries(pcd.points)

In [ ]:
import numpy as np

In [ ]:
np.save(exp_fpath / '60X_coords.npy',beads_60X_coords)

In [ ]:
np.save(exp_fpath / '40X_coords.npy',stitched_40X_coords)

# Visualize counts

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
fov = 10

In [ ]:
experiment_fpath = Path('processing_folder/LBEXP20210718_EEL_Mouse_448_2/fresh_tissue/')

In [ ]:
# Check fovs with no counts
fov_no_counts = []
for fov in np.arange(0,305):
    counts = pd.read_parquet(experiment_fpath / 'results' / ('20210718_124116_843__ChannelEuropium_Cy3_Seq0002_counts_beads_fresh_tissue_decoded_fov_' + str(fov) + '.parquet'))
    if counts.shape[0] <2:
        print(fov)
        fov_no_counts.append(fov)
    print(counts.shape)                                                    
                                            

In [ ]:
vw = napari.Viewer()

In [ ]:

import numpy as np
import scipy.ndimage as nd
from skimage import filters, morphology, measure
from skimage import img_as_float64
from pathlib import Path


# pysmFISH imports
from pysmFISH.io import load_raw_images
from pysmFISH.utils import convert_from_uint16_to_float64
from pysmFISH.logger_utils import selected_logger

from pysmFISH.dots_calling import osmFISH_peak_based_detection_fast

In [ ]:

coords = data.loc[:, ['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
_ = vw.add_points(coords)

In [ ]:
_ = vw.add_points(beads_org_tiles.tile_corners_coords_pxl,size=100)

In [ ]:
fov_subdataset = beads_data.dataset.loc[beads_data.dataset.fov_num == fov,: ].iloc[0]

In [ ]:
fpath = experiment_fpath / '20210718_124116_843__ChannelEuropium_Cy3_Seq0002_img_data.zarr'

In [ ]:
import zarr

storage = zarr.DirectoryStore(fpath)
root = zarr.group(store=storage,overwrite=False)


In [ ]:
io.load_raw_images()

In [ ]:
img = root['20210718_124116_843__ChannelEuropium_Cy3_Seq0002_fov_77']['raw_data_fov_77'][...]

In [ ]:
vw = napari.Viewer()
_ = vw.add_image(img)

In [ ]:
FlatFieldKernel=[100,100]
img = convert_from_uint16_to_float64(img)
img = img.max(axis=0)
img -= filters.gaussian(img,FlatFieldKernel,preserve_range=False)
img[img<0] = 0



In [ ]:
group = beads_grpd_fovs.get_group(77).iloc[0]

In [ ]:
vw = napari.Viewer()
_ = vw.add_image(img)

In [ ]:
parameters = {}
parameters['CountingFishMinObjDistance'] = 20
parameters['CountingFishMaxObjSize'] = 200
parameters['CountingFishMinObjSize'] = 20
parameters['CountingFishNumPeaksPerLabel'] = 1


In [ ]:
counts = osmFISH_peak_based_detection_fast(img,group,parameters)

In [ ]:
coords = counts.loc[:, ['r_px_original','c_px_original']].to_numpy()

In [ ]:
vw = napari.Viewer()
_ = vw.add_image(img)
_= vw.add_points(coords,size=5,opacity=0.7,symbol='+',face_color='magenta')

In [ ]:
dataset = pd.read_parquet(experiment_fpath / '211013_15_59_21_20210718_124116_843__ChannelEuropium_Cy3_Seq0002_img_data_dataset.parquet')

In [ ]:
beads_grpd_fovs = dataset.groupby('fov_num')

In [ ]:
group = beads_grpd_fovs.get_group(1).iloc[0]

In [ ]:
round_num = group.round_num

In [ ]:
round_num

In [ ]:
root.tree()

In [ ]:
fpath